In [1]:
import os
import sys
import numpy as np
sys.path.insert(0, os.path.realpath('../'))

In [2]:
import LDAQ

In [3]:
# here we should show how to use the visualization object:
# - use of functions on channels of subplot
    # - use of built in functions
    # - use of custom functions
# - refresh rates, t_span, x_lim, y_lim
# - use case add_image, add_lines
# ...

In [4]:
# Create simulated acquisition sources:
acq_simulated = LDAQ.simulator.SimulatedAcquisition(acquisition_name='sim')

In [5]:
def simulate_signal(t, f1, f2):
    import numpy as np
    sig1 = np.sin(2*np.pi*f1*t) + np.random.rand(len(t))*0.3
    sig2 = np.cos(2*np.pi*f2*t) + np.random.rand(len(t))*0.3
    return np.array([sig1, sig2]).T

acq_simulated.set_simulated_data(simulate_signal, channel_names=["ch1", "ch2"], sample_rate=10000, args=(84, 120))

In [6]:
def virtual_channel_func(self, ch1):
    try:
        # At virtual channel definition, retrieving data or channel indices is not yet possible for all channels.
        # Therefore, we use a try-except block to properly execute channel definition.
        time, data = self.get_data(N_points=2)
        i_ch3 = self.get_channel_index("ch3", channel_type='data') # retrieve data index channel of ch1
        ch3_prev = data[-1, i_ch3] # retrieve last value of ch1
        
    except:
        # at channel definition, no data is available yet. Therefore, we set the previous value to 0.
        ch3_prev = 0
        
    # cumulative sum of ch1:
    ch1_cumsum = np.cumsum(ch1) + ch3_prev	
    return ch1_cumsum.reshape(-1,1)

acq_simulated.add_virtual_channel("ch3", ["ch1"], function=virtual_channel_func)

In [7]:
def custom_vis_function(self, data):
    # this function is applied to each channel of the subplot individually.
    # data is a 1D array.
    
    return data**3 + 0.5

vis = LDAQ.Visualization(sequential_plot_updates=False)
vis.add_lines((0,0), source='sim', channels=["ch1", "ch2"], refresh_rate=50)
vis.add_lines((0,1), source='sim', channels=["ch1", "ch2"], refresh_rate=500, function="fft")
vis.add_lines((1,0), source='sim', channels=["ch1", "ch2"], refresh_rate=50, function=custom_vis_function)
vis.add_lines((1,0), source='sim', channels=["ch3"], refresh_rate=50)

vis.config_subplot((0,0), t_span=0.04, ylim=(-1.1, 1.1))
vis.config_subplot((0,1), t_span=0.5, ylim=(0, 1.1), xlim=(0, 500), rowspan=2)
vis.config_subplot((1,0), t_span=0.04)

In [8]:
ldaq = LDAQ.Core(acquisitions=[acq_simulated], visualization=vis)
ldaq.run(3.)

In [ ]:
ldaq.get_measurement_dict()['sim']['data'].shape

(3000000, 3)